In [135]:
import requests
import json
import pandas as pd
from datetime import datetime

### Set the time for Now

In [136]:
now = datetime.utcnow()

### Get bearer token

In [137]:
username = "timothy_woodland@outlook.com"
password = "@j-tLtrR8F"
client_secret = 'wZaRN7rpjn3FoNyF5IFuxg9uMzYJcvOoQ8QWiIqS3hfk6gLhVlG57j5YNoZL2Rtc'

In [138]:
payload = {'client_id':'tado-web-app',
            'grant_type':'password',
            'scope':'home.user',
            'username':username,
            'password':password,
            'client_secret':client_secret}

In [139]:
token_r = requests.post('https://auth.tado.com/oauth/token', params=payload)
token = token_r.json()["access_token"]
# print(token)

### Set Header for future Requests

In [140]:
header = {'Authorization' : "Bearer " + token}

### Get Home ID

In [141]:
home_id_r = requests.get("https://my.tado.com/api/v1/me", headers = header)
home_id = home_id_r.json()['homeId']
# print(home_id)

### Get name of home

In [142]:
home_url = "https://my.tado.com/api/v2/homes/"+str(home_id)
home_r = requests.get(home_url , headers = header)

In [143]:
home = home_r.json()["name"]

In [144]:
print(home)

Leslie Road


### Get zone data

In [145]:
zones_list_url = home_url+"/zones"

In [146]:
zone_list_r = requests.get(zones_list_url , headers = header)

In [147]:
zone_list_json = zone_list_r.json()
# print(json.dumps(zone_list_json, indent=1))

In [148]:
zone_list = []
zone_dict = {}
counter = 1


for zone_data in zone_list_json:
    zone_id = zone_data["id"]
    zone_name = zone_data["name"]

    zone_list.append(zone_id)
    
    zone_dict[zone_id] = {"time":str(now), "zone":zone_name}

    counter = counter + 1

### Get data from each zone and put into a dataframe

In [149]:
# for zone in zone_list:
#     zone_url = zones_list_url+"/"+str(zone)+"/state"
#     zone_r = requests.get(zone_url, headers = header)
#     zone_json = zone_r.json()

#     mode = zone_json["tadoMode"]
#     geo_override = zone_json["geolocationOverride"]
#     type = zone_json["setting"]["type"]
#     status = zone_json["setting"]["power"]
#     set_temp = zone_json["setting"]["temperature"]["celsius"]
#     power_level = zone_json["activityDataPoints"]["heatingPower"]["percentage"]
#     zone_temp = zone_json["sensorDataPoints"]["insideTemperature"]["celsius"]
#     zone_humidity = zone_json["sensorDataPoints"]["humidity"]["percentage"]

#     zone_data_dict = {"mode":mode, "geo_override":geo_override, "type":type, "status":status, "set_temp":set_temp, "power_level":power_level, "zone_temp":zone_temp, "zone_humidity":zone_humidity }

#     zone_dict[zone].update(zone_data_dict)
    

In [150]:
# zone_df = pd.DataFrame.from_dict(zone_dict, orient="index")

In [151]:
# zone_df = zone_df.reset_index()
# zone_df= zone_df.rename(columns={"index":"zone_id"})

In [152]:
# zone_df = zone_df.set_index("time")

### Testing Get data from a Zone

In [153]:
zone = 3

In [154]:
zone_url = zones_list_url+"/"+str(zone)+"/state"

In [155]:
zone_r = requests.get(zone_url , headers = header)

In [156]:
zone_json = zone_r.json()

In [157]:
print(zone_json)

{'tadoMode': 'HOME', 'geolocationOverride': False, 'geolocationOverrideDisableTime': None, 'preparation': None, 'setting': {'type': 'HEATING', 'power': 'OFF', 'temperature': None}, 'overlayType': 'MANUAL', 'overlay': {'type': 'MANUAL', 'setting': {'type': 'HEATING', 'power': 'OFF', 'temperature': None}, 'termination': {'type': 'TIMER', 'typeSkillBasedApp': 'NEXT_TIME_BLOCK', 'durationInSeconds': 1800, 'expiry': '2023-12-16T20:00:00Z', 'remainingTimeInSeconds': 17448, 'projectedExpiry': '2023-12-16T20:00:00Z'}}, 'openWindow': None, 'nextScheduleChange': {'start': '2023-12-16T20:00:00Z', 'setting': {'type': 'HEATING', 'power': 'ON', 'temperature': {'celsius': 18.0, 'fahrenheit': 64.4}}}, 'nextTimeBlock': {'start': '2023-12-16T20:00:00.000Z'}, 'link': {'state': 'ONLINE'}, 'activityDataPoints': {'heatingPower': {'type': 'PERCENTAGE', 'percentage': 0.0, 'timestamp': '2023-12-16T15:07:44.857Z'}}, 'sensorDataPoints': {'insideTemperature': {'celsius': 20.0, 'fahrenheit': 68.0, 'timestamp': '20

In [158]:
# mode = zone_json["tadoMode"]
# geo_override = zone_json["geolocationOverride"]
# type = zone_json["setting"]["type"]
# status = zone_json["setting"]["power"]
# set_temp = zone_json["setting"]["temperature"]["celsius"]
# power_level = zone_json["activityDataPoints"]["heatingPower"]["percentage"]
# zone_temp = zone_json["sensorDataPoints"]["insideTemperature"]["celsius"]
# zone_temp = zone_json["sensorDataPoints"]["humidity"]["percentage"]

In [159]:
zone_temp = zone_json["sensorDataPoints"]["humidity"]["percentage"]

In [160]:
print(json.dumps(zone_r.json(), indent=2))

{
  "tadoMode": "HOME",
  "geolocationOverride": false,
  "geolocationOverrideDisableTime": null,
  "preparation": null,
  "setting": {
    "type": "HEATING",
    "power": "OFF",
    "temperature": null
  },
  "overlayType": "MANUAL",
  "overlay": {
    "type": "MANUAL",
    "setting": {
      "type": "HEATING",
      "power": "OFF",
      "temperature": null
    },
    "termination": {
      "type": "TIMER",
      "typeSkillBasedApp": "NEXT_TIME_BLOCK",
      "durationInSeconds": 1800,
      "expiry": "2023-12-16T20:00:00Z",
      "remainingTimeInSeconds": 17448,
      "projectedExpiry": "2023-12-16T20:00:00Z"
    }
  },
  "openWindow": null,
  "nextScheduleChange": {
    "start": "2023-12-16T20:00:00Z",
    "setting": {
      "type": "HEATING",
      "power": "ON",
      "temperature": {
        "celsius": 18.0,
        "fahrenheit": 64.4
      }
    }
  },
  "nextTimeBlock": {
    "start": "2023-12-16T20:00:00.000Z"
  },
  "link": {
    "state": "ONLINE"
  },
  "activityDataPoints":

In [161]:
json_test = zone_r.json()

In [162]:
json_test["sensorDataPoints"]["insideTemperature"]["celsius"]

20.0

In [163]:
json_test["sensorDataPoints"]["humidity"]["percentage"]

62.4

### Test Weather

In [164]:
weather_url = "https://my.tado.com/api/v2/homes/"+str(home_id)+"/weather"

In [165]:
r = requests.get(weather_url , headers = header)

In [166]:
# print(json.dumps(r.json(), indent=2))

In [167]:
test_json = r.json()

In [168]:
json_df = pd.DataFrame.from_dict(test_json)

In [169]:
test_json["weatherState"]["value"]

'CLOUDY'

In [170]:
test_json["outsideTemperature"]["celsius"]

10.86

### Get Weather data and put into a dataframe

In [171]:
weather_json = r.json()

In [172]:
weather_data = [weather_json["outsideTemperature"]["celsius"], weather_json["weatherState"]["value"], "tado"]

In [173]:
weather_columns = ["temperature_C", "state", "source"]

In [174]:
weather_df = pd.DataFrame(data=[weather_data], index=[now], columns=weather_columns)